## 0. Imports/ Defining Helper Fuctions 

In [3]:
from __future__ import print_function
import pandas as pd
import requests
from requests_oauthlib import OAuth1
import cnfg
import tweepy
from tweepy.models import Status, ResultSet
import re
from datetime import datetime
import random
import preprocessor as p
import re
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import spacy
import nltk
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore
from gensim import corpora, models, similarities, matutils
import gensim
import pyLDAvis
import pyLDAvis.gensim
from collections import Counter
from gensim.corpora.dictionary import Dictionary
#nlp = spacy.load('en')
import pickle
import pandas as pd
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk_stopwords = stopwords.words("english")+["health",'healthy','fitness','food','...','dont','follow','good','new','good','giving','get','like','share','week','pron','please','today','day','monday','sunday',"week","fit","amp",'february',"-pron-","rt", "via","-»","--»","--","---","-->","<--","->","<-","«--","«","«-","»","«»"]

In [49]:
config = cnfg.load(".twitter_config")

oauth = OAuth1(config["consumer_key"],
               config["consumer_secret"],
               config["access_token"],
               config["access_token_secret"])

auth = tweepy.OAuthHandler(config["consumer_key"],
                           config["consumer_secret"])
#auth = tweepy.AppAuthHandler(config["consumer_key"],
#                           config["consumer_secret"])

#auth.set_access_token(config["access_token"],
#                      config["access_token_secret"])

api=tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [23]:
def search_twitter(query, number):
	"""
	Argument Order: query, number

	Will search twitter for the query. Query can be a list.
	Number relates to how many tweets

	Returns a list of tweets
	"""
	assert type(query) == str, "Please enter a query in the form of a string"
	assert type(number) == int, "Please enter the number of as an integer"

	return list(tweepy.Cursor(api.search, q=query, lang='en', tweet_mode='extended').items(number))

def extract_handle(tweet):
	"""
	Argument Order: tweet

	Extracts the twitter handle for a given tweet. @ symbol not included.

	Returns the handle - string type
	"""
	assert type(tweet) == Status, "Please enter in a tweet of type Status"

	return tweet.__dict__['user'].screen_name

def extract_text(tweet):
	"""
	Argument Order: tweet

	Extracts the clean text of a tweet. Remove links and emoji's
	Returns clean text of the tweet
	"""

	#this function can be mapped to a list of tweets (status type)

	assert type(tweet) == Status, "Please enter in a tweet of type Status"

	regex = r"http\S+"
	subset = ""

	emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

	if hasattr(tweet, "text"):
		clean = re.sub(regex, subset, tweet.text.strip())
		clean = emoji_pattern.sub(subset, clean).strip()
	else:
		clean = re.sub(regex, subset, tweet.full_text.strip())
	return clean



def extract_hashtags(tweet):
	"""
	Argument Order: tweet

	Return a list of hastags present in a given tweet
	"""
	hashtags = []
	assert type(tweet) == Status, "Please enter in a tweet of type Status"

	if hasattr(tweet, "entities"):
		if tweet.entities['hashtags'] == []:
			return []
		else:
			for i in tweet.entities['hashtags']:
				hashtags.append(i['text'])
	else:
		print("No entity method!")
	return hashtags


def extract_datetime(tweet):
	"""
	Argument Order: tweet

	Returns a datetime object
	"""
	assert type(tweet) == Status, "Please enter in a tweet of type Status"

	return tweet.created_at

def extract_users_tweets(handle, number):
	"""
	Argument Order: handle, number of tweets to extract
	
	Extract's a user's tweets
	"""
	final = ResultSet() #can change to resultset later if I want

	try:
		for status in tweepy.Cursor(api.user_timeline, screen_name=handle, count=200, include_rts=True).items(number):
			final.append(status)
	except:
		print("{} is a protected user!")
		return []

	return final

def tweet_cleaner(tweet_text):
    clean_tweets = []
    for text in tweet_text:
        clean_tweets.append(p.clean(text))

    import string
    translator = str.maketrans('', '', string.punctuation)

    tweets = []
    for text in clean_tweets:
        tweets.append((str(text).translate(translator)))  

    tweet = []
    for words in tweets:
        tweet.append(words.lower())

    wordnet_lemmatizer = WordNetLemmatizer()
    final_tweets = []
    for words in tweet:
        final_tweets.append(wordnet_lemmatizer.lemmatize(words))
    return(final_tweets)

## 1. Gathering Tweets/Extracting Features & Cleaning Tweet Text

In [4]:
with open('nutrition_tweets.pkl', 'rb') as f:
    nutrition_tweets = pickle.load(f)

In [5]:
with open('health_tweets.pkl', 'rb') as f:
    health_tweets = pickle.load(f)

In [8]:
hashtags = []

for tweet in nutrition_tweets:
    hashtags.append(extract_hashtags(tweet))

tweet_text = []

for tweet in nutrition_tweets:
    tweet_text.append(extract_text(tweet))
    
date_time = []

for tweet in nutrition_tweets:
    date_time.append(extract_datetime(tweet))
    
handle = []

for tweet in nutrition_tweets:
    handle.append(extract_handle(tweet))
    
location = []
for tweet in nutrition_tweets:
    location.append(tweet.place)

In [9]:
hashtags2 = []

for tweet in health_tweets:
    hashtags2.append(extract_hashtags(tweet))

tweet_text2 =[]

for tweet in health_tweets:
    tweet_text2.append(extract_text(tweet))
    
date_time2 = []

for tweet in health_tweets:
    date_time2.append(extract_datetime(tweet))
    
handle2 = []

for tweet in health_tweets:
    handle2.append(extract_handle(tweet))
    
location2 = []
for tweet in health_tweets:
    location2.append(tweet.place)

#### Build Dataframe (if not already) from Extracted Tweets/Features 

In [ ]:
my_tweets = search_twitter('nutrition',10000)

Rate limit reached. Sleeping for: 820


In [10]:
df = pd.DataFrame()
df["text"] = tweet_text
df["datetime"]= date_time
df["location"] = location
df["hashtags"] = hashtags
df["handle"] = handle

In [11]:
df2 = pd.DataFrame()
df2["text"] = tweet_text2
df2["datetime"]= date_time2
df2["location"] = location2
df2["hashtags"] = hashtags2
df2["handle"] = handle2

Joining our two dataframes of tweets

In [12]:
frames = [df,df2]

In [13]:
df_health = pd.concat(frames)

In [38]:
tweet_text = df2['text']

In [40]:
df2.head()

,text,datetime,location,hashtags,handle
0,RT @Notorious_Nava: The #Athletic #MMA #Fitnes...,2018-03-04 02:15:33,None,"[Athletic, MMA, Fitness, Judo, Fight, Diet, fi...",BlackTigerSport
1,RT @adventureista: Happy #Holi! May the canvas...,2018-03-04 02:15:29,None,"[Holi, love, health]",Exluppo
2,#health #beauty Buy Now: $49.99 6 Pair Andrea ...,2018-03-04 02:15:28,None,"[health, beauty]",health_anbeauty
3,Permanent health results are only achieved whe...,2018-03-04 02:15:21,None,"[healthyliving, habits, health]",MariStreamlined
4,RT @wef: Scientists have created a silicon bea...,2018-03-04 02:15:09,None,[health],SammKttkrr


In [41]:
tweets = list(map(lambda x: x.replace("#", ""), tweet_text))
tweet_text = list(map(lambda x: x.replace("&amp;", "&"), tweets))
tweets = list(map(lambda x: x.replace("&gt;", "greater than"), tweet_text))
tweet_text = list(map(lambda x: x.replace("&lt;", "less than"), tweets))
tweets = list(map(lambda x: x.replace("&amp;", "&"), tweet_text))
tweet_text = list(map(lambda x: x.replace("/", ""), tweets))
tweets = list(map(lambda x: x.replace("...", ""), tweet_text))
tweet_text = list(map(lambda x: x.replace("\u200d", ""), tweets))
tweets = list(map(lambda x: x.replace("\U0001f986", ""), tweet_text))
tweet_text = list(map(lambda x: x.replace("\U0001f942", ""), tweets))
tweets = list(map(lambda x: x.replace("\U0001f92f", ""), tweet_text))
tweet_text = list(map(lambda x: x.replace("\U0001f911", ""), tweets))
tweets = list(map(lambda x: x.replace("$", ""), tweet_text))
tweet_text = list(map(lambda x: x.replace("n't", ""), tweets))
tweets = list(map(lambda x: x.replace("ç", "c"), tweet_text))
tweet_text = list(map(lambda x: x.replace("'ll", ""), tweets))

In [42]:
final_tweets = tweet_cleaner(tweet_text)

## 2. Topic Modeling with LDA/NMF and Word2Vec

In [43]:
count_vectorizer = CountVectorizer(min_df = 10, ngram_range=(1, 2), max_df=.5,
                                   stop_words=nltk_stopwords, token_pattern="\\b[a-z][a-z]+\\b")
count_vectorizer.fit(final_tweets)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=10,
        ngram_range=(1, 2), preprocessor=None,
        stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs',...ron-', 'rt', 'via', '-»', '--»', '--', '---', '-->', '<--', '->', '<-', '«--', '«', '«-', '»', '«»'],
        strip_accents=None, token_pattern='\\b[a-z][a-z]+\\b',
        tokenizer=None, vocabulary=None)

In [44]:
counts = count_vectorizer.transform(final_tweets).transpose()

In [45]:
id2word = dict((v, k) for k, v in count_vectorizer.vocabulary_.items())

In [46]:
corpus = matutils.Sparse2Corpus(counts)

In [47]:
lda = models.LdaModel(corpus=corpus, num_topics=15, id2word=id2word, minimum_probability=.03, passes=10)

/Users/hector/anaconda/lib/python3.6/site-packages/gensim/matutils.py:995: RuntimeWarning: invalid value encountered in subtract
  result = psi(alpha) - psi(np.sum(alpha))


KeyboardInterrupt: 

In [ ]:
lda.print_topics()